In [1]:
import pandas as pd
import numpy as np

In [12]:
train_data = pd.read_csv('./Yes_Bank_Training.csv')
test_data = pd.read_csv('./Yes_Bank_Test.csv')

In [13]:
train_data = train_data.drop(axis=1, columns=['days_passed', 'campaign_contacts', 'month_of_year', 'date'])
test_data = test_data.drop(axis=1, columns=['days_passed', 'campaign_contacts', 'month_of_year', 'date'])

# Remove unknown.

In [ ]:
def replace_unknown(col, replace):
    print(train_data[col].unique())
    ind = []
    for index, row in train_data.iterrows():
        if row[col] == 'unknown':
            ind.append(index)
    
    print(len(ind))
    for index in ind:
        train_data.at[ind,col]=replace

    print(train_data[col].unique())

From 'job_description'.

In [ ]:
replace_unknown('job_description','blue-collar')

From 'education_details'

In [14]:
replace_unknown